# Parameter fiting using emcee

This notebook computes the confidence regions of a set of cosmological parameters using Cosmic Chronometes, Megamasers, Supernovae Ia. This code employs Markov Chain Monte Carlo (MCMC) using the `emcee` package.


To improve computation speed, we use `multiprocessing`, which enables parallel processing of complex functions by leveraging multiple CPU cores.


The code is designed for the `Base+Union3` data. If you want to change the SNe data to Pantheon+ or DES, make sure to update the `likelihood.chi2_Base_Union3` in the minimization process and in the `lnposterior(theta)` function to the corresponding `chi2_Base_PantheonPlus` or `chi2_Base_DES`, along with the prior value for the absolute magnitude.

In [ ]:
import likelihood
import numpy as np
import emcee
import corner
#from multiprocessing import Pool                                # Use this for Linux 
from pathos.multiprocessing import ProcessingPool as Pool        # Use this for MacOS
import scipy.optimize as opt

In this part, we minimize the $\chi^{2}$ function to find the parameters values that will serve as initial points in the MCMC sampling. This ensures the Markov chains start near the region of highest likelihood, improving convergence efficiency.  

In [ ]:
seed_Union3=np.array([ 3.4728e+03,  1.0181e+04,  7.80e+03,  8.448e+03,
        7.04511e+03,  5.38222e+02,  3.3014e-01,  7.0811e+01,
       -7.3189e-01, -1.27761e+00,  6.4713e-02,  1.3984e+02])

minimum=opt.minimize(likelihood.chi2_Base_Union3,seed_Union3,method="Nelder-Mead")
print(minimum)

Here, we define the priors for the parameters, except for the megamasers velocities, which are treated as nuisance parameters. Additionally, we define the logarithm of the posterior distribution, which will be used as input for the `emcee`.  

In [ ]:
def lnprior(theta):
    v1,v2,v3,v4,v5,v6,Om0,H0,w0,wa,M,rd=theta
    if 0.<Om0<1. and 0.<H0<150. and -2<w0<-1/3 and -5<wa<5  and -1<M<1 and 0<rd<250:
        return 0
    return -np.inf

def lnposterior(theta):
    lnposterior=0
    prior =lnprior(theta)
    if prior == 0:
        lnposterior=-likelihood.chi2_Base_Union3(theta)/2 
    else:
        return(prior)
    if np.isnan(lnposterior):
        print("Theres a problem with this parameters",theta)
    return lnposterior

We initialize the `emcee` walkers by randomly sampling around the values that minimize the $\chi^{2}$ function, ensuring a good starting point for the MCMC. We also execute `emcee` to generate the Markov chains and perform the sampling.  

In [ ]:
ndim,nwalkers=12,25
p1=2*np.random.rand(nwalkers,ndim)-1
scale=np.array([10,10,10,10,10,10,0.1,10,0.1,0.1,0.1,10])
p2=minimum.x
initial_points=p2+np.multiply(p1,scale)

nsteps=20000   
with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnposterior, pool=pool)
    sampler.run_mcmc(initial_points,nsteps,progress=True)

Finally, we obtain the chains from the sampler and generate contour plots of the parameters using `corner`, which helps visualize the posterior distributions and the correlations between the different parameters.  

In [ ]:
samples=sampler.get_chain()
tau = sampler.get_autocorr_time()

flat_samples = sampler.get_chain(discard=int(nsteps/3),thin=int(np.min(tau)/2),flat=True)

fig = corner.corner(
    flat_samples,labels=["v1","v2","v3","v4","v5","v6","Om0","H0","w0","wa","M","rd"],plot_contours=True,plot_datapoints=False,show_titles=True,smooth=True)